# Exemple 10.2
## Calcul de la prime d'un produit sur deux têtes

Un homme (x) de 55 ans et sa femme (y) de 50 ans ont acheté une police de rente différée. 

Si un des deux meurt dans les 10 premières années, un capital décès de 200 000 \\$ est payable au survivant à la fin de l'année. 

Si les deux survivent aux 10 années, ils obiennent une rente de 50 000 \\$ par année, payable mensuellement jusqu'a la mort d'un des deux conjoints, où le montant est réduit à 30 000 \\$ par année. Au moment du décès du dernier survivant, la rente cesse.

Les primes pour se produits sont nivelées et sont payables mensuellement pour au plus 10 ans si seulement les deux sont vivants.


Le taux d'intérêt effectif annuel est de 5\% et la mortalité suit le Standard Select Survival Model pour les deux conjoints.
1. La mortalité de (55) et (50) sont indépendantes.
2. (55) et (50) sont *select* au moment de l'achat de la police.

### Préambule

In [1]:
# Fonction pour créer un lien entre un age (clé) et une lx (valeur)

generate_hashmap <- function(keys, values){
    hashmap <- function(index){
        key_index <- match(index, keys)
        return (values[key_index])
    }
    return (Vectorize(hashmap))
}

In [2]:
interpolate_death <- function(lives, ages, periodicity){
    death_by_ages <- lives(tail(ages,-1))-lives(head(ages,-1))
    death_by_month <- rep(death_by_ages , each=periodicity) / periodicity
    return(death_by_month)
}

project_interpolated_lives <- function(lives_at_age1, interpolated_death){
    cumulative_death_since_age1 <- cumsum(interpolated_death)
    return(lives_at_age1+c(0,cumulative_death_since_age1))
}


In [3]:
i = 0.05

### 4 produits a évaluer

#### Prime
1. Une rente mensuelle immediate payable pendant 10 ans ou jusqu'au premier décès  

#### Prestation
2. Une assurance vie payable si le status vie conjointe termine avant 10 ans
3. Une rente différée 10 ans sur 65:60 payable jusqu'au **premier** décès
4. Une rente différée 10 ans sur 65:60 payable jusqu'au **dernier** décès


### Développement

#### Produit #1 (Prime): Rente mensuelle immediate payable pendant 10 ans ou jusqu'au premier décès 

prime = P * E[rente]

$$ \begin{align}
\mathrm{E[rente]} &= \frac{1}{12}\big(\sum^{10*12}_{t=0} v^{\frac{t}{12}} \,_{\frac{t}{12}}p_{[55]:[50]}\big) \\
\mathrm{Par \; independance}\\
\mathrm{E[rente]} &= \frac{1}{12}\big(\sum^{10*12}_{t=0} v^{\frac{t}{12}} \,_{\frac{t}{12}}p_{[55]}\,_{\frac{t}{12}}p_{[50]}\big) \\
\end{align}  
$$

In [4]:
# Calcul des probabilité aux ages entiers - Données tirées de (Appendix D, table 1)

age_x <- 55:65
lx_select55_par_age <- c(97807.07,
                         97640.4,
                         97435.17,
                         97195.56,
                         96929.59,
                         96634.14,
                         96305.75,
                         95940.6,
                         95534.43,
                         95082.53,
                         94579.73)

age_y <- 50:60
lx_select50_par_age <- c(98552.51, 
                         98450.67, 
                         98326.19, 
                         98181.77, 
                         98022.38, 
                         97846.2, 
                         97651.21, 
                         97435.17, 
                         97195.56, 
                         96929.59, 
                         96634.14)

lx <- generate_hashmap(age_x, lx_select55_par_age)
ly <- generate_hashmap(age_y, lx_select50_par_age)

In [36]:
# Evaluation des probabilité interpolées mensuellement (via distribution uniforme des décès)
# On suppose que les décès pour chacuns des groupes sélect-ultime sont uniformes,non pas les décès conjoints
# C'est une approximation adéquate selon AMLCR

m <- 12

interpolated_lives_x <- project_interpolated_lives(lx(55), interpolate_death(lx, age_x, m))
interpolated_lives_y <- project_interpolated_lives(ly(50), interpolate_death(ly, age_y, m))

tp55 <- head(interpolated_lives_x, -1) / lx(55)
tp50 <- head(interpolated_lives_y, -1) / ly(50)


In [37]:
# Valeur de la rente

# Index des pas des temps
t_prime <- seq(0, 10*12-1)

# Valeur actualisées
v_prime <- (1+i)^-(t_prime/12)

valeur_produit_1 <- sum(v_prime * tp55 * tp50)/12

cat("réponse = P", round(valeur_produit_1, 4))

réponse = P 7.7782

#### Produit #2: Capital décès en cas de décès d'un des membre du couple dans les 10 premières années

Valeur = 200 000 * E[assurance]

$$ \begin{align}
\mathrm{E[assurance]} &= \sum^{10-1}_{t=0} v^{t+1} \,_t|q_{[55]:[50]} \\
\mathrm{E[assurance]} &= \sum^{10-1}_{t=0} v^{t} (Pr[t < T_{xy} \leq t+1]) \\
\mathrm{E[assurance]} &= \sum^{10-1}_{t=0} v^{t} (Pr[T_{xy} > t] - Pr[T_{xy} > t+1]) \\
\mathrm{Par \; independance}\\
\mathrm{E[assurance]} &= \sum^{10-1}_{t=0} v^{t} (Pr[T_x > t]Pr[T_y>t]] - Pr[T_x > t+1]Pr[T_y > t+1]) \\
\mathrm{E[assurance]} &= \sum^{10-1}_{t=0} v^{t} (\,_tp_{[55]}\,_tp_{[50]}-\,_{t+1}p_{[55]}\,_{t+1}p_{[50]}) \\
\end{align}  
$$

In [72]:
t_produit_2 = 1:10

# Valeur actualisées
v_produit_2 = (1+i)^-t_produit_2

# Calcul des probabilité du survie indivuduelles 
tp55 = lx(age_x)/lx(55)
tp50 = ly(age_y)/ly(50)

t_barre_q_xy = (head(tp50, -1) * head(tp55, -1) - tail(tp55, -1) * tail(tp50, -1))

esperance_assurance = sum(v_produit_2 * t_barre_q_xy)
valeur_produit_2 = 200000 * round(esperance_assurance,4)

cat("réponse = ", valeur_produit_2)

[1] 11

réponse =  7660

#### Produit #3 et Produit #4: Rente immediate différée décroissante au premier décès

##### Valeur à l'émission

$$ \begin{align}
\mathrm{Valeur\;emission} &= v^{10} \,_tp_{[50]}\; \,_tp_{[50]} \mathrm{Valeur\;prestations}\\
\end{align}
$$


##### Valeur au début des prestations de rente

$$ \begin{align}
\mathrm{Valeur\;prestations} &= 30\,000 \;\mathrm{E[rente]}^{(12)}_{\overline{65:60}} + 20\,000\;\mathrm{E[rente]}^{(12)}_{65:60}\\
\mathrm{Valeur\;prestations} &= 30\,000\;(\; \mathrm{E[rente]}^{(12)}_{65}+\mathrm{E[rente]}^{(12)}_{60}-\mathrm{E[rente]}^{(12)}_{65:60}\;) + 20\,000\;\mathrm{E[rente]}^{(12)}_{65:60}\\
\mathrm{Valeur\;prestations} &= 30\,000\;\mathrm{E[rente]}^{(12)}_{65} + 30\,000\;\mathrm{E[rente]}^{(12)}_{60} - 10\,000\; \mathrm{E[rente]}^{(12)}_{65:60}
\end{align}
$$


In [39]:
# Calcul des probabilités aux âges entiers - Données tirées de (Appendix D, table 1)

age_ultime <- 60:101
lx_ultime_par_age <- c(96634.14,
                       96305.75,
                       95940.6,
                       95534.43,
                       95082.53,
                       94579.73,
                       94020.33,
                       93398.05,
                       92706.06,
                       91936.88,
                       91082.43,
                       90133.96,
                       89082.09,
                       87916.84,
                       86627.64,
                       85203.46,
                       83632.89,
                       81904.34,
                       80006.23,
                       77927.35,
                       75657.16,
                       73186.31,
                       70507.19,
                       67614.6,
                       64506.5,
                       61184.88,
                       57656.68,
                       53934.73,
                       50038.65,
                       45995.64,
                       41841.05,
                       37618.56,
                       33379.88,
                       29183.78,
                       25094.33,
                       21178.3,
                       17501.76,
                       14125.89,
                       11102.53,
                       8469.73,
                       6248.17,
                       0)

l_ultime <- generate_hashmap(age_ultime, lx_ultime_par_age)

In [40]:
# Calcul des probabilité de survie mensuelles
m <- 12

interpolated_lives_60 <- project_interpolated_lives(l_ultime(60), interpolate_death(l_ultime, 60:101, m))
interpolated_lives_65 <- project_interpolated_lives(l_ultime(65), interpolate_death(l_ultime, 65:101, m))

t_mois_p60 <- interpolated_lives_60 / l_ultime(60)
t_mois_p65 <- interpolated_lives_65 / l_ultime(65)

###### Calcul de la rente de (60)

In [ ]:
t_60 <- seq(0,length(t_mois_p60)-1)
v_60 <- (1+i)^(-t_60/12)
rente_60 <- sum(v_60*t_mois_p60)/12

###### Calcul de la rente de (65)

In [59]:
t_65 <- seq(0,length(t_mois_p65)-1)
v_65 <- (1+i)^(-t_65/12)
rente_65 <- sum(v_65*t_mois_p65)/12

###### Calcul de la rente vie conjointe

In [68]:
t_mois_p60_tronque <- head(t_mois_p60, length(t_mois_p65)) # comme le reste de t_mois_p65 sont des 0 par la suite!
t_p_vc <- t_mois_p60_tronque * t_mois_p65
t_vc <- seq(0,length(t_mois_p65)-1)
v_vc <- (1+i)^(-t_65/12)
rente_vc <- sum(v_vc * t_p_vc)/12

##### Calcul de la valeur des prestatations à l'émission

In [73]:
valeur_prestations = 30000*rente_65 + 30000*rente_60 - 10000*rente_vc

valeur_produit_3_4 = valeur_prestations * (1+i)^-10 * tp55[10] * tp55[10]

#### Calcule de la prime

$$ \begin{align}
P \; \mathrm{produit}_1 &= \mathrm{produit}_2 + \mathrm{produit}_{3,4} \\
P &= \frac{\mathrm{produit}_2 + \mathrm{produit}_{3,4}}{\mathrm{produit}_1}\\
\end{align}
$$

In [46]:
prime = round((valeur_produit_2+valeur_produit_3_4)/valeur_produit_1/12, 2)
prime

[1] 4468.34